# Calculating the absorption and scattering cross section using a probability distribution

In [1]:
import time
# AA = time.time()
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as spit
import os
print('hello world')


hello world








### First, we specify which dust we are using

In [2]:
dust_dir = ['/home/physics/Research/DUSTY/DUSTY/Lib_nk/', 
            "C:/UTSA/Research/DUSTY/DUSTY/Lib_nk/"]
# this is the possible locations of where dust can be


nk_path = dust_dir[1]               #where the dust is 
dust = 'oliv_nk_y.nk'                  #DUST NAME HERE #grf
rho = 3.33 #grams cm**-3            #density
pathy = os.path.join(nk_path, dust) #pipeline is open
wavelen, n_dust, k_dust = np.loadtxt(pathy, skiprows=12, unpack=True)
                                    #lamda, n, and k values are extracted
m = np.array([complex(n_dust[i], k_dust[i]) for i in range(len(wavelen))])
                                    #joins n, k values into complex number
    
wavelen = wavelen**(-1) * 10000   #Convert wavelen2 to waveLENGTH from waveNUMBER
print('hello world')


hello world


### This defines the probability distribution

In [13]:
def probability(dis_name, l1, l2, lmin=0.05, m1=0, m2=0, d=0):
    '''
    This is the probability distribution as a function of L1 and L2, the 
    geometric parameters. This parameter gets inserted into the integral that 
    calculates the average polarizability per unit volume

    Parameters
    ----------
    dis_name : String
        This specifies the distribution we will be using. 
        'CDE' = Continuous Distribution of Ellipsoids
        'ERCDE' = Externally Restricted CDE, returns CDE if lmin=0
        'tCDE' = truncated CDE, REQUIRES MORE WORK
    l1 : Float
        Largest Geometric Constant, lmin<l1<1.0
    l2 : Float
        Second Largest Geometric Constant, lmin<l2<=l1
    lmin : Float, optional
        Minimum allowed geometric constant.  The default is 0.

    Returns
    -------
    Float
        Function dependent on l1 and l2

    '''
    l3 = 1 - l1 - l2
    if dis_name == 'CDE':
        return 2
    elif dis_name == 'CDE2':
        return 120 * l1 * l2 * l3
    elif dis_name == 'ERCDE':
        return 2/((1 - (3*lmin))**2)
    elif dis_name == 'tCDE':
        return 1/((1-d-m2)*(1-m1-m2-d) - 0.5*((1-d-m2)**2) - m1**2)
        
    
print('hello world')

hello world


### Now we define our volume function. v_avg is used to calculate sigma below

In [4]:
def volume_integrand_mrn(r, q):
    v = r**(-q)
    return v


rmin = 0.005
rmax = 0.25
q = 3.5

r_integral = spit.quad(volume_integrand_mrn, rmin, rmax, args=q)
r_average = ((1/(rmax - rmin)) * r_integral[0])**(1/-q)
v_avg = (4./3.) * np.pi * r_average**3



### creates function that calculates Sigma, defined in eq 13 in Min et al 2003

In [5]:
def sigma(m, lamda, v):
    sig = []
    for i in range(len(lamda)):
        k = (2.0 * np.pi)/lamda[i]
        term1 = (6.0*np.pi) / (v * (k**3))
        term2 = np.imag((m[i]**2))
        term3 = 1.0 / abs(m[i]**2 - 1)**2
        sig.append(term1 * term2 * term3)
    return sig
print('hello world')

hello world


### creates our bounds for our geometric factors. The bounds are the sides of a triangle in (l1, l2) space

In [6]:
def bounds_l1():
    return [0,1]

def bounds_l2(l1):
    return [0,1-l1]
print('hello world')


hello world


### This is where we calculate the absorption cross-section (Cabs). It creates an empty list, then calculates Cabs for a CDE distribution at each wavelength as described in Min 03, eqn 15. It then uses this to find the shape averaged mass absorption coefficient for particles of a given volume, as described in Min 03, eqn 39

In [7]:
aa = time.time()
cabs_cde = []
for j in range(len(m)):
    def f(l1, l2, n=m[j], dis_name='CDE'):
        b = 1/(n**2 - 1)
        term1 = 1/3 * 1/(b + l1)
        term2 = 1/3 * 1/(b + l2)
        term3 = 1/3 * 1/(b + 1 - l1 - l2)
        # r = np.real((term1 + term2 + term3)*probability(dis_name, l1, l2))
        j = np.imag((term1 + term2 + term3)*probability(dis_name, l1, l2))
        return j
        # return np.real((term1 + term2 + term3)*probability(dis_name, l1, l2)) + np.imag((term1 + term2 + term3)*probability(dis_name, l1, l2))
    cabs_cde.append(spit.nquad(f, [bounds_l2, bounds_l1])[0])
kappa_cde = np.array((cabs_cde))
kappa_cde *= (2 * np.pi / (1e-4*wavelen)) * v_avg

bb = time.time()
print('This took {} seconds to run'.format(bb-aa))

This took 50.385767698287964 seconds to run


### Same as above, using CDE2 distribution

In [8]:
aa = time.time()
cabs_cde2 = []
for j in range(len(m)):
    def f(l1, l2, n=m[j], dis_name='CDE2'):
        b = 1/(n**2 - 1)
        term1 = 1/3 * 1/(b + l1)
        term2 = 1/3 * 1/(b + l2)
        term3 = 1/3 * 1/(b + 1 - l1 - l2)
        # r = np.real((term1 + term2 + term3)*probability(dis_name, l1, l2))
        j = np.imag((term1 + term2 + term3)*probability(dis_name, l1, l2))
        return j
        # return np.real((term1 + term2 + term3)*probability(dis_name, l1, l2)) + np.imag((term1 + term2 + term3)*probability(dis_name, l1, l2))
    cabs_cde2.append(spit.nquad(f, [bounds_l2, bounds_l1])[0])

kappa_cde2 = np.array((cabs_cde2))
kappa_cde2 *= (2 * np.pi / (1e-4*wavelen)) * v_avg
bb = time.time()
print('This took {} seconds to run'.format(bb-aa))

This took 46.05778884887695 seconds to run


### Same as above, using ERCDE distribution

In [9]:
aa = time.time()
cabs_ercde = []
for j in range(len(m)):
    def f(l1, l2, n=m[j], dis_name='ERCDE'):
        b = 1/(n**2 - 1)
        term1 = 1/3 * 1/(b + l1)
        term2 = 1/3 * 1/(b + l2)
        term3 = 1/3 * 1/(b + 1 - l1 - l2)
        # r = np.real((term1 + term2 + term3)*probability(dis_name, l1, l2))
        j = np.imag((term1 + term2 + term3)*probability(dis_name, l1, l2))
        return j
        # return np.real((term1 + term2 + term3)*probability(dis_name, l1, l2)) + np.imag((term1 + term2 + term3)*probability(dis_name, l1, l2))
    cabs_ercde.append(spit.nquad(f, [bounds_l2, bounds_l1])[0])

kappa_ercde = np.array((cabs_ercde))
kappa_ercde *= (2 * np.pi / (1e-4*wavelen)) * v_avg
bb = time.time()
print('This took {} seconds to run'.format(bb-aa))

This took 50.91991567611694 seconds to run


### Spheres only

In [16]:
cabs_spheres = []
for j in range(len(m)):
    cabs_spheres.append((2 * np.pi / (1e-4*wavelen[j]))*v_avg*np.imag(3*(m[j]**2 - 1)/(m[j]**2 + 2)))
cabs_spheres

[6.851976886835575e-07,
 8.9522205599989e-07,
 1.1334623236485478e-06,
 1.3945932374245906e-06,
 1.687119829511351e-06,
 2.024106978528954e-06,
 2.3734363379485935e-06,
 2.750422434548647e-06,
 3.155001253719803e-06,
 3.5871493069163997e-06,
 4.070455070852882e-06,
 4.558984386888086e-06,
 5.074858564338689e-06,
 5.645900344336081e-06,
 6.2178078191075966e-06,
 6.816899325933141e-06,
 7.475101990013158e-06,
 8.163089612445405e-06,
 8.846151908851139e-06,
 9.592147940744393e-06,
 1.0330315072447867e-05,
 1.1133974003281817e-05,
 1.1966959208398576e-05,
 1.282926320116488e-05,
 1.3720685484643133e-05,
 1.4641225415176853e-05,
 1.5590661667396873e-05,
 1.6568870393660756e-05,
 1.7575862726811205e-05,
 1.8611384204011157e-05,
 1.9726271813479592e-05,
 2.0819781053623594e-05,
 2.1995219161914872e-05,
 2.3146323143510677e-05,
 2.4381369189953175e-05,
 2.5646916457766533e-05,
 2.6942836082584907e-05,
 2.826853523407492e-05,
 2.9624039692939192e-05,
 3.100916858568659e-05,
 3.2487942054887985e

### Here is where everything culminates. It gives us our required data in a table that can be input into DUSTY

In [10]:
Cabs_array = np.array((cabs_cde))
sig = np.array((sigma(m, wavelen, v_avg)))
Csca_array = Cabs_array/sig

output = np.transpose((wavelen, Cabs_array, Csca_array))


## This is where the text file is created with it's appropriate header

In [11]:
line0 = f'standard mrn mixture (a_min = {rmin}, amax = {rmax})\n'
# line1 = f'dust, weight: {dust[:-3]}, {wt_a}; {dust2[:-3]}, {wt_b}; {dust3[:-3]}, {wt_c}\n'
line2 = ' lambda \t <C_abs>/v \t <C_sca>/v\n'

f = open('look at this.dat', 'w')
f.write(line0)
# f.write(line1)
f.write(line2)
# f.write(str(output))


for i in range(len(output)):
    f.write(f"{output[i,0]} \t {output[i,1]} \t {output[i,2]}\n")


f.close()


In [12]:
print('hello world')

hello world
